- This notebook is to compare some summary statistics between oma releases (number and size of OMA Groups, HOGs, added or removed genomes/taxa levels, number of homoeolog pairs, etc)
- By: Natasha Glover
- Last updated: 
   - Natasha: 13 Aug 2018
   - Adrian:  12 Jul 2019
   - Adrian:  04 Jan 2021


# Setup

In [ ]:
# Scientific libraries
import numpy as np
#from scipy import stats
import pandas as pd
import tables
import pandas

#basic python libraries
import re
import os
from datetime import datetime

# Pyoma libraries
from pyoma.browser import sanitychecks

# Graphic libraries
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

## Aesthetics

In [ ]:
# Extra options
pd.set_option('max_colwidth',200)
pd.options.display.max_rows = 150

#style options
%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12.0, 8.0)

#seaborn options
sns.set(rc={'axes.facecolor':'white', 'figure.facecolor':'white'}, font_scale=1.5)
sns.set_style('whitegrid')
palette="husl"

## Mounting the oma browser directory

To compare oma browser releases, I use the hdf5 file from each. However, since they are so big (~50 Gb each), I mount the directory on vital-it where they are contained. This is done with sshfs and needs to be mounted before the following analyses. The following command is done from command line:
- sshfs nglover@prd.vital-it.ch:/data/ul/projects/cdessimo/oma-browser ~/oma_mount

## Parameters

TODO: Make it so that it just detects all the databases automatically and then we can run the notebook automatically after each release.

In [ ]:
#directory containing the different oma browser releases
oma_browser_dir = os.path.expanduser(os.getenv("DARWIN_BROWSER_SHARE", "~/oma_mount/"))
prefix = "All."
cand_releases = [z for z in os.listdir(oma_browser_dir)
                 if z.startswith(prefix) and os.path.isdir(os.path.join(oma_browser_dir, z))]
releases = sorted(cand_releases, key=lambda x: datetime.strptime(x, prefix+"%b%Y"))[-4:]
print(releases)

#list of 4 releases (strings) to be compared (only exactly 4 for now)
#note they will be plotted and presented in the order specified in list
#releases = ["All.Dec2018", "All.Jun2019", "All.Aug2020", "All.Apr2021"]

# Running the sanity checks

The following code does all the heavy lifting and uses the sanitychecks2 library from pyoma. Since this involves making many computations, especially from the Entries table in each database, this could take a while to run (~40 minutes).

In [ ]:
#import importlib
#importlib.reload(sanitychecks)

#list of sanity (release) objects, each one corresponding to a different release
sanity_sessions = []

for release in releases:
    release_obj = sanitychecks.SanitySession(oma_browser_dir, release)
    sanity_sessions.append(release_obj)

# Functions

In [ ]:
#TODO add these functions to the sanity checks module

In [ ]:
def _return_group_object(release_obj, group):
    if group == "hog":
        group_obj = release_obj.all_hogs
    if group == "omagroup":
        group_obj = release_obj.omagroups
    return group_obj

def count_nb_groups(release_obj, group):
    df = pd.DataFrame.from_dict(_return_group_object(release_obj, group), orient='index')
    df['release'] = release_obj.release    
    return df.groupby('release').size().item()

def get_nb_genes_per_group_df(release_obj, group):
    df = pd.DataFrame.from_dict(_return_group_object(release_obj, group), orient='index')
    df['release'] = release_obj.release
    df.reset_index(inplace=True) 
    df.rename({0:"nb_genes", "index":group}, inplace=True, axis=1)    
    return df

def sort_df_by_release(df):
    rel_dtype = pd.CategoricalDtype(categories=releases, ordered=True)
    df.release = df.release.astype(rel_dtype)
    df.sort_values('release', inplace=True)
    return df

def cum_bin_cnts_by_release(df, column, bin_width=10):
    """return the cumulative counts of the number of groups that have
    at least the size of the bin value"""
    bins = np.arange(0, max(df[column])+bin_width, bin_width)
    dfq = pandas.DataFrame({rel: np.histogram(df[df['release']==rel][column], bins)[0] 
                            for rel in df['release'].unique()})
    cum = dfq.iloc[::-1].cumsum()
    cum['bin'] = bins[:-1][::-1]
    cum['lab'] = np.array(list(">{}".format(z) for z in bins[:-1]))[::-1]
    return cum

def make_countplot_by_release(df, releases, title="Don't forget a title"):
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12,8))
    ax = sns.countplot(x="release", data=df, order=releases, palette="husl")
    ax.set_title(title)
    return ax

def get_differences_in_genomes_or_taxa(df, old_release, new_release, genomes_or_taxa):
    '''old_release and new_release should be sanity session objects'''
    
    if genomes_or_taxa == "genomes":
        added = list(set(new_release.species) - set(old_release.species))
        removed = list(set(old_release.species) - set(new_release.species))
        
    if genomes_or_taxa == "taxa":
        added = list(set(new_release.all_hog_lvls) - set(old_release.all_hog_lvls))
        removed = list(set(old_release.all_hog_lvls) - set(new_release.all_hog_lvls))
        added = [x.decode("utf-8") for x in added]
        removed = [x.decode("utf-8") for x in removed]
    res = {'added':added, 'removed': removed}
    return res

def decode_column_in_df(df, column_to_decode):
    df[column_to_decode] = df.apply(lambda x: x[column_to_decode].decode("utf-8"), axis=1)
    return df

def make_boxplot_by_release(df, releases, column_to_plot, title="Don't forget a title"):
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(14,4))
    sns.boxplot(x="release", y=column_to_plot, data=df, ax=ax[0], palette=palette, order=releases)
    ax[0].set_title("Outliers shown")
    sns.boxplot(x="release", y=column_to_plot, data=df, ax=ax[1], showfliers=False, palette=palette, order=releases)
    ax[1].set_title("No outliers shown")
    plt.suptitle(title)
    plt.subplots_adjust(top=0.75)
    plt.show()
    
def make_cumbiggerthan_plot_by_release(cum_df, releases, title="Don't forget a title", xlabel="Size", ylabel="Cumulative counts"):
    pal = matplotlib.colors.ListedColormap(sns.color_palette(palette, len(releases)))
    fig, ax = plt.subplots()
    for num, rel in enumerate(releases):
        plt.plot('bin', rel, data=cum_df, marker='', color=pal(num), linewidth=2.4, alpha=0.9, label=rel)
    plt.semilogy()
    xstep = max(1,len(cum_df)//20)
    if cum_df['bin'][0]>0:
        xstep = -xstep
    plt.xticks(cum_df['bin'][::xstep], fontsize=15, rotation=45)
    plt.legend()
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title, fontsize=15)
    plt.show()
    
def get_prop_with_n_genes(df, releases, nb_genes=2):
    '''get number of groups per release
    param releases is the list of string releases'''
    df1 = df.groupby('release').size().reset_index()
    df1.rename({0:'total_nb_groups'}, axis=1, inplace=True)

    #get groups with n genes
    df2 = df[df['nb_genes']==nb_genes].groupby('release').size().reset_index()
    df2.rename({0:'nb_groups_with_'+str(nb_genes)+'_genes'}, axis=1, inplace=True)
    
    #get proportion
    df3 = pd.merge(left=df1, right=df2, how="left", on="release")
    df3['prop_groups_'+str(nb_genes)+'_genes'] = df3['nb_groups_with_'+str(nb_genes)+'_genes']/ df3['total_nb_groups']
    df3.set_index("release", inplace=True)
    df3 = df3.reindex(releases)
    df3.reset_index(level=0, inplace=True)
    return df3

def get_total_nb_groups(df):
    df = df.groupby('release').size().reset_index()
    df.rename({0:"nb_genomes"}, axis=1, inplace=True)
    df = sort_df_by_release(df) 
    return df

def get_summary_stats(df, column_to_summarize):
    df= df[['release',column_to_summarize]].groupby('release').describe().reset_index()
    df.columns = df.columns.droplevel()
    df = df.rename({'':'release'}, axis=1)
    df = sort_df_by_release(df)
    return df

def plot_proportions_of_groups_with_n_genes(low_genes_df, list_of_columns_to_plot,\
                                            title="Don't forget a title", rename_cols=False):
    
    if rename_cols==True:
        low_genes_df = rename_proportion_columns(low_genes_df)
        list_of_columns_to_plot = rename_proportion_columns(low_genes_df).columns[1:5]
        
    
    #melt dataframe
    melted_low_genes_df = pd.melt(low_genes_df, id_vars=['release'], \
                                  value_vars=list_of_columns_to_plot)
    melted_low_genes_df.rename({"variable": "nb genes in group", "value": "proportion of groups"}, \
                               axis=1, inplace=True)
    
    #make plot
    fig, ax= plt.subplots(figsize=(10,10))
    sns.pointplot(x="release", y="proportion of groups", hue="nb genes in group", data=melted_low_genes_df, \
                 palette=palette)
    #plt.legend(loc="upper right")
    plt.ylim(0,.7)
    plt.title(title)
    plt.show()

def get_low_genes_df(df, releases, list_of_gene_numbers=[2,3,4,5]):
    '''makes a dataframe with proportion of genes with n number for each release.'''
    low_genes_df = pd.DataFrame.from_dict({'release': releases})

    for i in range(list_of_gene_numbers[0],list_of_gene_numbers[-1]+1):
        tmp_df = pd.DataFrame(get_prop_with_n_genes(df, releases, i))
        low_genes_df = pd.merge(left=low_genes_df, right=tmp_df[['release',tmp_df.columns[-1]]],\
                                how="left", on="release")

    low_genes_df['total_nb_groups'] = get_prop_with_n_genes(df, releases, 2)['total_nb_groups']
    return low_genes_df

def rename_proportion_columns(df):
    columns = df.columns
    new_columns = {}
    for column in columns:
        if "prop_groups" in column:
            matchObj = re.match(r'prop_groups_(.*)_genes', column)
            try:
                new_column = matchObj.group(1)
                new_columns[column] = new_column
            except:
                print("no match")
    
    df = df.rename(new_columns, axis=1)
    return df


# Compare Genomes

In [ ]:
#get genome dataframe
tmp_dfs = []
for release in sanity_sessions:
    tmp_df = pd.DataFrame.from_dict({'genomes': release.species})
    tmp_df['release'] = release.release
    tmp_dfs.append(tmp_df)
genome_df = pd.concat(tmp_dfs, ignore_index=True)
del tmp_dfs

## Nb of genomes

In [ ]:
get_total_nb_groups(genome_df)

In [ ]:
make_countplot_by_release(genome_df, releases, "Number of Genomes")

## Genomes added

In [ ]:
#latest genomes added
genome1_df = pd.DataFrame(sanity_sessions[-2].genome_table.read())
genome1_df = decode_column_in_df(genome1_df, "UniProtSpeciesCode")
genome2_df = pd.DataFrame(sanity_sessions[-1].genome_table.read())
genome2_df = decode_column_in_df(genome2_df, "UniProtSpeciesCode")

diff = get_differences_in_genomes_or_taxa(genome_df, sanity_sessions[-2], sanity_sessions[-1], "genomes")
genome2_df.loc[genome2_df['UniProtSpeciesCode'].isin(diff['added'])] 

In [ ]:
print("In total there were {:d} new genomes in the latest release".format(len(diff['added'])))

## Genomes removed

In [ ]:
genome1_df.loc[genome1_df['UniProtSpeciesCode'].isin(diff['removed'])] 

In [ ]:
print("In total there were {:d} removed genomes in the latest release".format(len(diff['removed'])))

# Compare number of proteins

In [ ]:
#note this include ASVs
df = pd.DataFrame([[r.release, int(r.entries_table.attrs.NROWS)] for r in sanity_sessions],
                  columns=['release', 'nr_entries'])
sort_df_by_release(df)

In [ ]:
sns.stripplot(x="release", y="nb_entries",data=df, order=releases, size=15, palette="husl")
plt.title("Number of Entries")
plt.ticklabel_format(style='plain', axis='y')

# Compare OMA Groups

In [ ]:
#get omagroups df
omagroups_df = pd.concat([get_nb_genes_per_group_df(release, "omagroup") for release in sanity_sessions], ignore_index=True)

## Nb of omagroups

In [ ]:
#total number of omagroups
get_total_nb_groups(omagroups_df)

In [ ]:
make_countplot_by_release(omagroups_df, releases, title="Total Number of OMA Groups")

## Nb genes per OMA Groups summary stats

In [ ]:
get_summary_stats(omagroups_df, "nb_genes")

## Plot distributions

In [ ]:
make_boxplot_by_release(omagroups_df, releases, "nb_genes","Number of genes per OMA Group")

In [ ]:
cum = cum_bin_cnts_by_release(omagroups_df, 'nb_genes')
make_cumbiggerthan_plot_by_release(cum, releases, 'Number of groups with at least x member genes')

## Groups with low numbers of genes

In [ ]:
low_genes_df = get_low_genes_df(omagroups_df, releases)
low_genes_df

In [ ]:
low_genes_df.rename({'prop_groups_2_genes':"2",'prop_groups_3_genes':"3",\
                     'prop_groups_4_genes':"4",'prop_groups_5_genes':"5"}, axis=1, inplace=True)


plot_proportions_of_groups_with_n_genes(low_genes_df, low_genes_df.columns[1:5],\
                                        "Proportion of OMA Groups with n number of genes")

# Compare HOGs (subhogs)

In [ ]:
#get hog df
fam_re = re.compile(rb'HOG:(?P<rel>[A-Z]?)(?P<fam>\d+)')
hog_df = pd.concat([get_nb_genes_per_group_df(release, "hog") for release in sanity_sessions], ignore_index=True)
hog_df['depth'] = hog_df['hog'].apply(lambda x: x.count(b'.'))
hog_df['fam'] = hog_df['hog'].apply(lambda x: int(fam_re.match(x).group('fam')))

## Nb of hogs

In [ ]:
#total number of hogs
get_total_nb_groups(hog_df)

In [ ]:
make_countplot_by_release(hog_df, releases, title="Total Number of HOGs")

## Nb genes per HOG summary stats

In [ ]:
get_summary_stats(hog_df, "nb_genes")

In [ ]:
get_summary_stats(hog_df, "depth")

## Plot distributions

In [ ]:
make_boxplot_by_release(hog_df, releases, "nb_genes", "Number of genes per HOG")

In [ ]:
cum_hog = cum_bin_cnts_by_release(hog_df, 'nb_genes', bin_width=10)
make_cumbiggerthan_plot_by_release(cum_hog, releases, 'Number of groups with at least x member genes')

In [ ]:
hog_depth = pd.DataFrame({rel: np.histogram(hog_df[hog_df['release']==rel]['depth'],bins=np.arange(0, max(hog_df['depth']+2), 1))[0] for rel in df['release'].unique()})
hog_depth['bin'] = np.arange(0, max(hog_df['depth']+1), 1)
make_cumbiggerthan_plot_by_release(hog_depth, releases, "Number of hogs with a certain nesting depth x", xlabel="hog depth", ylabel="Counts of HOGs")

## HOGs with low numbers

In [ ]:
low_genes_df = get_low_genes_df(hog_df, releases, [1,2,3,4])
low_genes_df

In [ ]:
plot_proportions_of_groups_with_n_genes(low_genes_df, low_genes_df.columns[1:5],\
                                        "Proportion of Subhogs with n number of genes",\
                                       rename_cols=True)

# Compare families (root hogs)

In [ ]:
fam_df = hog_df[hog_df['depth']==0]
fam_df.reset_index();

## Nb of families

In [ ]:
#total number of families
get_total_nb_groups(fam_df)

## Nb genes per fam summary stats

In [ ]:
get_summary_stats(fam_df, "nb_genes")

In [ ]:
make_countplot_by_release(fam_df, releases, title="Number of families")

## Plot distributions

In [ ]:
make_boxplot_by_release(fam_df, releases, "nb_genes","Number of genes per Family")

In [ ]:
cum_fam = cum_bin_cnts_by_release(fam_df, 'nb_genes')
make_cumbiggerthan_plot_by_release(cum_fam, releases, 'Number of groups with at least x member genes')

## Families with low numbers

In [ ]:
low_genes_df = get_low_genes_df(fam_df, releases, [2,3,4,5])
low_genes_df

In [ ]:
plot_proportions_of_groups_with_n_genes(low_genes_df, low_genes_df.columns[1:5],\
                                        "Proportion of Root Hogs with n number of genes",\
                                       rename_cols=True)


# Track HOGs for a few key genes

In [ ]:
# cases reported once by BASF
query_genes = ["P53_HUMAN", ]
# a few random ARATH genes
query_genes.extend(["PME34_ARATH", "CYSK1_ARATH", "TO202_ARATH", "RIBA1_ARATH"])
# opsin gene where OMA doesn't work well (until Aug2020 at least)
query_genes.extend(["ENSG00000102076"])
import pyoma.browser.db
data = []
for release in sanity_sessions:
    db = pyoma.browser.db.Database(release.db_path)
    for query in query_genes:
        try:
            members = db.member_of_fam(db.hog_family(db.ensure_entry(db.id_resolver.resolve(query))))
        except pyoma.browser.db.InvalidId:
            print("{} does not contain an xref to {}".format(release.release, query))
            members = []
        except pyoma.browser.db.Singleton:
            print("{} is a singleton in {}".format(query, release.release))
            members = [1]
        data.append((query, release.release, len(members)))
    db.close()
case_df = pd.DataFrame(data, columns=["gene", "release", "size"])

In [ ]:
fig, ax= plt.subplots(figsize=(10,10))
sns.pointplot(x="release", y="size", hue="gene", data=case_df, palette=palette)
plt.title('Roothog-size for a few families')
plt.show()

# Compare at different taxa levels

In [ ]:
tmp_dfs = []
for release in sanity_sessions:
    tmp_df = pd.DataFrame.from_dict(release.all_hog_lvls, orient="index")
    tmp_df = tmp_df.rename({0:'nb_hogs'}, axis=1)
    tmp_df = tmp_df.reset_index()
    tmp_df['release'] = release.release
    tmp_dfs.append(tmp_df)
tax_df = pd.concat(tmp_dfs, ignore_index=True)
del tmp_dfs

tax_df.rename({'index':'taxa'}, axis=1, inplace=True)
tax_df['taxa'] = tax_df.apply(lambda x: x['taxa'].decode("utf-8"), axis=1)

## Nb of taxa levels

In [ ]:
get_total_nb_groups(tax_df)

In [ ]:
#number of taxa
make_countplot_by_release(tax_df, releases, title="Number of taxonomic levels")

## Nb hogs per taxa summary stats

In [ ]:
#number of hogs per taxonomic level summary stats
get_summary_stats(tax_df, 'nb_hogs')

In [ ]:
#distribution of number of hogs per taxa level
make_boxplot_by_release(tax_df, releases, 'nb_hogs', "Number of HOGS per taxa")

## Taxa added

In [ ]:
old_release = sanity_sessions[-2]
new_release = sanity_sessions[-1]
get_differences_in_genomes_or_taxa(tax_df, old_release, new_release, "taxa")['added']

## Taxa removed

In [ ]:
get_differences_in_genomes_or_taxa(tax_df, old_release, new_release, "taxa")['removed']

## Taxa with 2 genes in family

The following analysis uses the latest release. TODO: THIS IS NOT TRUE I THINK

In [ ]:
#get families with 2 genes
families_2_genes = fam_df[(fam_df['nb_genes']==2) & (fam_df['release']==releases[3])]['fam']

#open up latest h5 file
with tables.open_file(oma_browser_dir+"/"+releases[3]+"/data/OmaServer.h5", "r") as h5file:
    #read hog level table and make df
    hog_level_df = pd.DataFrame(h5file.root.HogLevel.read())

In [ ]:
#get rows with rootlevel taxa matching list of families w/ 2 genes
taxa_2_genes_at_root_df = hog_level_df[hog_level_df['Fam'].isin(families_2_genes)].\
                                                            drop_duplicates(subset='Fam').\
                                                            groupby('Level').\
                                                            size().\
                                                            reset_index().\
                                                            sort_values(0, ascending=False)
taxa_2_genes_at_root_df = taxa_2_genes_at_root_df.rename({0:"nb_families"}, axis=1)
taxa_2_genes_at_root_df = decode_column_in_df(taxa_2_genes_at_root_df, "Level")

In [ ]:
#top 20 taxa with the most families consisting of 2 genes
taxa_2_genes_at_root_df[:20]


# Compare Cross-references

In [ ]:
tmp_dfs = []
for session in sanity_sessions:
    tmp_df = session.xref_df
    tmp_df['release'] = session.release
    tmp_dfs.append(tmp_df)
xref_df = pd.concat(tmp_dfs, ignore_index=True)
del tmp_dfs

## Total number of Crossreferences in dataset

In [ ]:
sns.barplot(data=xref_df.groupby(['release'])['Counts'].sum().reset_index(),
            x="release", y="Counts", order=releases, palette="husl")

## Nr of xrefs per source

We can distringuish between the different sources of cross-references. In general the number of cross-references should be more or less stable accross the different releases.

In [ ]:
ax = sns.barplot(data=xref_df.groupby(['release', "Source"])['Counts'].sum().reset_index(),
             x="release", y="Counts", hue="Source", palette="gist_earth", order=releases)
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

## Nr of xrefs per genome

In [ ]:
ax = sns.barplot(data=xref_df.groupby(["release", "Species"])["Counts"].sum().reset_index(),
                x="release", y="Counts", hue="Species", palette="gist_earth", order=releases)
ax.legend(bbox_to_anchor=(-0.2, -0.12), ncol=7, loc='upper left', borderaxespad=0.)

The above picture is not easy to read, but it shows the spread of crossreferences among the different species. More interpretable is the following figure. It shows the relative change in the number of crossreferences per species between the last two releases. Negative values mean that there are fewer crossreference in the newer release, positive indicate the oposite

In [ ]:
old = sanity_sessions[-2].xref_df.groupby(['Species'])['Counts'].sum()
new = sanity_sessions[-1].xref_df.groupby(['Species'])['Counts'].sum()
rel_diff = ((new-old)/old).reset_index()
rel_diff.rename(columns={'Counts':"Rel_diff"}, inplace=True)
ax = sns.scatterplot(data=rel_diff, x="Species", y="Rel_diff")
plt.xticks(rotation=90, fontsize=9)
ax.set_ylabel("Relative difference between {} and {}".format(releases[-2], releases[-1]));


In [ ]:
sign_drop = rel_diff[rel_diff['Rel_diff'] < -0.15]
if len(sign_drop) > 0.2 * len(rel_diff):
    print("WARNING: {:.1f}% of all species have at least 15% less crossreferences!!".format(100*len(sign_drop)/len(rel_diff)))
print("In {} release, {} species have fewer crossreference than in {} release".format(releases[-1], len(rel_diff[rel_diff['Rel_diff']<0]), releases[-2]))

## GO annotations

In [ ]:
go_df = pd.DataFrame([(rel.release, len(rel.h5_handle.root.Annotations.GeneOntology)) for rel in sanity_sessions],
                     columns=["release", "Annotations"])
sns.barplot(data=go_df, x="release", y="Annotations", order=releases, palette="husl")

## EC annotations

In [ ]:
ec_df = pd.DataFrame([(rel.release, len(rel.h5_handle.root.Annotations.EC)) for rel in sanity_sessions],
                     columns=["release", "Annotations"])
sns.barplot(data=ec_df, x="release", y="Annotations", order=releases, palette="husl")

## Domain Annotations

In [ ]:
dom_df = pd.DataFrame([(rel.release, len(rel.h5_handle.root.Annotations.Domains)) for rel in sanity_sessions],
                     columns=["release", "Annotations"])
sns.barplot(data=dom_df, x="release", y="Annotations", order=releases, palette="husl")